In [1]:
#Import Play Tennis Data
import pandas as pd
df=pd.read_csv("PlayTennis.csv")
print(df)

     Outlook Temperature Humidity    Wind Play Tennis
0      Sunny         Hot     High    Weak          No
1      Sunny         Hot     High  Strong          No
2   Overcast         Hot     High    Weak         Yes
3       Rain        Mild     High    Weak         Yes
4       Rain        Cool   Normal    Weak         Yes
5       Rain        Cool   Normal  Strong          No
6   Overcast        Cool   Normal  Strong         Yes
7      Sunny        Mild     High    Weak          No
8      Sunny        Cool   Normal    Weak         Yes
9       Rain        Mild   Normal    Weak         Yes
10     Sunny        Mild   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
13      Rain        Mild     High  Strong          No


In [2]:
# Cell 2: Define entropy function
def entropy(probs):
    import math
    return sum(-prob * math.log(prob, 2) for prob in probs)


In [3]:
# Cell 3: Define entropy_of_list function to calculate entropy of target attribute
def entropy_of_list(a_list):
    from collections import Counter
    cnt = Counter(x for x in a_list)
    print(cnt)
    num_instances = len(a_list)
    probs = [x / num_instances for x in cnt.values()]
    print(num_instances)
    print(probs)
    return entropy(probs)

total_entropy = entropy_of_list(df['Play Tennis'])
print(total_entropy)


Counter({'Yes': 9, 'No': 5})
14
[0.35714285714285715, 0.6428571428571429]
0.9402859586706309


In [4]:
# Cell 4: Define information_gain function
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    df_split = df.groupby(split_attribute_name)
    print(df_split)
    for name, group in df_split:
        print("Name", name)
        print("Group", group)
        nobs = len(df.index) * 1.0
        print(nobs)
        print("NOBS", nobs)
        df_agg_ent = df_split.agg({target_attribute_name: [entropy_of_list, lambda x: len(x) / nobs]})[target_attribute_name]
        avg_info = sum(df_agg_ent['entropy_of_list'] * df_agg_ent['<lambda_0>'])
        old_entropy = entropy_of_list(df[target_attribute_name])
        return old_entropy - avg_info


In [5]:
# Cell 5: Define id3DT function (ID3 Decision Tree)
def id3DT(df, target_attribute_name, attribute_names, default_class=None):
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt) == 1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class = max(cnt.keys())
        # print("attributes_names:", attribute_names)
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names]
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_names[index_of_max]
        tree = {best_attr: {}}
        remaining_attributes_names = [i for i in attribute_names if i != best_attr]
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3DT(data_subset, target_attribute_name, remaining_attributes_names, default_class)
            tree[best_attr][attr_val] = subtree
        return tree


In [6]:
# Cell 6: Run the ID3 and display the results
attribute_names = list(df)
print("List of Attributes:", attribute_names)
attribute_names.remove('Play Tennis')  # Remove the class attribute
print("Predicting Attributes:", attribute_names)

from pprint import pprint
tree = id3DT(df, 'Play Tennis', attribute_names)
print("The Resultant Decision Tree is ")
pprint(tree)

attribute = next(iter(tree))
print("Best Attribute: \n", attribute)
print("Tree Keys\n ", tree[attribute].keys())


List of Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Tennis']
Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']
Name Overcast
Group      Outlook Temperature Humidity    Wind Play Tennis
2   Overcast         Hot     High    Weak         Yes
6   Overcast        Cool   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
14.0
NOBS 14.0
Counter({'Yes': 4})
4
[1.0]
Counter({'Yes': 3, 'No': 2})
5
[0.6, 0.4]
Counter({'No': 3, 'Yes': 2})
5
[0.6, 0.4]
Counter({'Yes': 9, 'No': 5})
14
[0.35714285714285715, 0.6428571428571429]
Name Cool
Group     Outlook Temperature Humidity    Wind Play Tennis
4      Rain        Cool   Normal    Weak         Yes
5      Rain        Cool   Normal  Strong          No
6  Overcast        Cool   Normal  Strong         Yes
8     Sunny        Cool   Normal    Weak         Yes
14.0
NOBS 14.0
Counter({'Yes': 3, 'No': 1})
4
[0.75, 0.25]
Counter({'No': 2,